In [1]:
#Import Libraries
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from urduhack.preprocessing import remove_punctuation
from urduhack.preprocessing import normalize_whitespace
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt to C:\Users\Karakorum
[nltk_data]     Traders\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\Karakorum Traders\.conda\envs\tf\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
#Input Dataset
df = pd.read_csv('News_Urdu_Dataset.csv')
print("Headline = ",df.shape)

Headline =  (21, 3)


In [39]:
mydata = df.Headline
data = mydata.tolist()
sentence = ' '.join(data)

In [40]:
print(len(sentence))

1241


In [41]:
#Word Tokenization

tokenized_word=word_tokenize(sentence)
print(tokenized_word)

['عالمی', 'بینک', 'عسکریت', 'پسندی', 'سے', 'متاثرہ', 'خاندانوں', 'کی', 'معاونت', 'کرے', 'گا', 'مالی', 'سال', '2020', 'ریٹرن', 'فائل', 'کرنے', 'والوں', 'کی', 'تعداد', 'میں', '23', 'فیصد', 'کمی', 'جاپان', 'کو', 'سندھ', 'کے', 'خصوصی', 'اقتصادی', 'زون', 'میں', 'سرمایہ', 'کاری', 'کی', 'دعوت', 'برامدات', '767', 'فیصد', 'بڑھ', 'کر', 'ارب', '16', 'کروڑ', 'ڈالر', 'سے', 'زائد', 'کے', 'الیکٹرک', 'کو', 'اضافی', 'بجلی', 'گیس', 'کی', 'فراہمی', 'کے', 'قانونی', 'تقاضے', 'تعطل', 'کا', 'شکار', 'کھانے', 'پینے', 'کی', 'اشیا', 'کی', 'قیمتیں', 'سال', 'کی', 'بلند', 'ترین', 'سطح', 'پر', 'پہنچ', 'گئیں', 'اقوام', 'متحدہ', 'صنعتی', 'صارفین', 'کیلئے', 'بجلی', 'کے', 'پیک', 'ور', 'ٹیرف', 'اسکیم', 'کا', 'خاتمہ', 'اور', 'سبسڈی', 'منظور', 'پاکستان', 'میں', 'موبائل', 'کمپنیاں', 'مقامی', 'طور', 'پر', 'اسمبلنگ', 'کی', 'جانب', 'گامزن', 'گنے', 'کی', 'بروقت', 'کٹائی', 'سے', 'چینی', 'کی', 'قیمت', 'کم', 'ہو', 'کر', '85', 'روپے', 'فی', 'کلوگرام', 'تک', 'پہنچ', 'گئی', 'نومبر', 'میں', 'مہنگائی', 'معمولی', 'کمی', 'سے', '83', 'فیص

In [42]:
import stanza
stanza.download('urdu')
nlp = stanza.Pipeline('ur')

2023-06-13 15:27:22 INFO: "urdu" is an alias for "ur"
2023-06-13 15:27:22 INFO: Downloading default packages for language: ur (Urdu) ...
2023-06-13 15:27:23 INFO: File exists: C:\Users\Karakorum Traders\stanza_resources\ur\default.zip
2023-06-13 15:27:25 INFO: Finished downloading models and saved to C:\Users\Karakorum Traders\stanza_resources.
2023-06-13 15:27:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-06-13 15:27:26 INFO: Loading these models for language: ur (Urdu):
| Processor | Package |
-----------------------
| tokenize  | udtb    |
| pos       | udtb    |
| lemma     | udtb    |
| depparse  | udtb    |

2023-06-13 15:27:26 INFO: Using device: cpu
2023-06-13 15:27:26 INFO: Loading: tokenize
2023-06-13 15:27:26 INFO: Loading: pos
2023-06-13 15:27:27 INFO: Loading: lemma
2023-06-13 15:27:27 INFO: Loading: depparse
2023-06-13

In [43]:
doc = nlp(df.Headline[20])
doc

[
  [
    {
      "id": 1,
      "text": "وائرس",
      "lemma": "وائرس",
      "upos": "NOUN",
      "xpos": "NN",
      "feats": "Case=Acc|Gender=Masc|Number=Sing|Person=3",
      "head": 3,
      "deprel": "nmod",
      "start_char": 0,
      "end_char": 5
    },
    {
      "id": 2,
      "text": "کے",
      "lemma": "کا",
      "upos": "ADP",
      "xpos": "PSP",
      "feats": "AdpType=Post|Case=Acc|Gender=Masc|Number=Sing",
      "head": 1,
      "deprel": "case",
      "start_char": 6,
      "end_char": 8
    },
    {
      "id": 3,
      "text": "پھیلا",
      "lemma": "پھیلا",
      "upos": "NOUN",
      "xpos": "NN",
      "feats": "Case=Acc|Gender=Masc|Number=Sing|Person=3",
      "head": 11,
      "deprel": "obl",
      "start_char": 9,
      "end_char": 14
    },
    {
      "id": 4,
      "text": "میں",
      "lemma": "میں",
      "upos": "ADP",
      "xpos": "PSP",
      "feats": "AdpType=Post",
      "head": 3,
      "deprel": "case",
      "start_char": 15,
      "end

In [44]:
#Stop Words

stopwords = ['لگے', 'ہیں', 'کی', 'کرنے', 'کو', 'آپ', 'میرا', 'کرو', 'کوئی', 'ہوا', 'ہونے', 'والا', 'تھا', 'رہتا', 'آیا', 'نہ', 'گئی', 'و', 'یہ','مجھے', 'رہی', 'نہیں', 'رہا', 'میں', 'ہوں', 'ہے', 'میرے', 'اپنی', 'کچھ', 'کر', 'میری', 'ہو', 'سے', 'رہے', 'کے', 'لیے', 'کا', 'اپنے', 'نے', 'ایک', 'گیا']

In [46]:
# removing the stop words

word_tokens = word_tokenize(sentence)

tokenized_word = []

for w in word_tokens:
    if w not in stopwords:
        tokenized_word.append(w)

In [47]:
tokenized_word

['عالمی',
 'بینک',
 'عسکریت',
 'پسندی',
 'متاثرہ',
 'خاندانوں',
 'معاونت',
 'کرے',
 'گا',
 'مالی',
 'سال',
 '2020',
 'ریٹرن',
 'فائل',
 'والوں',
 'تعداد',
 '23',
 'فیصد',
 'کمی',
 'جاپان',
 'سندھ',
 'خصوصی',
 'اقتصادی',
 'زون',
 'سرمایہ',
 'کاری',
 'دعوت',
 'برامدات',
 '767',
 'فیصد',
 'بڑھ',
 'ارب',
 '16',
 'کروڑ',
 'ڈالر',
 'زائد',
 'الیکٹرک',
 'اضافی',
 'بجلی',
 'گیس',
 'فراہمی',
 'قانونی',
 'تقاضے',
 'تعطل',
 'شکار',
 'کھانے',
 'پینے',
 'اشیا',
 'قیمتیں',
 'سال',
 'بلند',
 'ترین',
 'سطح',
 'پر',
 'پہنچ',
 'گئیں',
 'اقوام',
 'متحدہ',
 'صنعتی',
 'صارفین',
 'کیلئے',
 'بجلی',
 'پیک',
 'ور',
 'ٹیرف',
 'اسکیم',
 'خاتمہ',
 'اور',
 'سبسڈی',
 'منظور',
 'پاکستان',
 'موبائل',
 'کمپنیاں',
 'مقامی',
 'طور',
 'پر',
 'اسمبلنگ',
 'جانب',
 'گامزن',
 'گنے',
 'بروقت',
 'کٹائی',
 'چینی',
 'قیمت',
 'کم',
 '85',
 'روپے',
 'فی',
 'کلوگرام',
 'تک',
 'پہنچ',
 'نومبر',
 'مہنگائی',
 'معمولی',
 'کمی',
 '83',
 'فیصد',
 'ڈسکوز',
 'بجلی',
 'ٹیرف',
 '86',
 'پیسے',
 'اضافے',
 'مطالبہ',
 'جوہری',
 'بجلی',
 'گھر',
 

In [48]:
#Before and After Stop Words Removing

print(f"Before {len(word_tokens)} and  after {len(tokenized_word)} words in the vocabulary") 

Before 257 and  after 194 words in the vocabulary


In [49]:
def text_cleaning(e):
  remove_punc = normalize_whitespace(remove_punctuation(e))
  return [word for word in remove_punc.split() if word not in stopwords]

In [52]:
print(df.Headline.apply(text_cleaning))

0     [عالمی, بینک, عسکریت, پسندی, متاثرہ, خاندانوں,...
1     [مالی, سال, 2020, ریٹرن, فائل, والوں, تعداد, 2...
2     [جاپان, سندھ, خصوصی, اقتصادی, زون, سرمایہ, کار...
3     [برامدات, 767, فیصد, بڑھ, ارب, 16, کروڑ, ڈالر,...
4     [الیکٹرک, اضافی, بجلی, گیس, فراہمی, قانونی, تق...
5     [کھانے, پینے, اشیا, قیمتیں, سال, بلند, ترین, س...
6     [صنعتی, صارفین, کیلئے, بجلی, پیک, ور, ٹیرف, اس...
7     [پاکستان, موبائل, کمپنیاں, مقامی, طور, پر, اسم...
8     [گنے, بروقت, کٹائی, چینی, قیمت, کم, 85, روپے, ...
9               [نومبر, مہنگائی, معمولی, کمی, 83, فیصد]
10         [ڈسکوز, بجلی, ٹیرف, 86, پیسے, اضافے, مطالبہ]
11    [جوہری, بجلی, گھر, زمائش, کیلئے, ایندھن, لوڈنگ...
12    [پاک, سوزوکی, اور, انڈس, موٹرز, گاڑیوں, قیمتوں...
13    [نومبر, ریونیو, وصولی, 346, ارب, روپے, ساتھ, ہ...
14       [نیشنل, بینک, تقرریوں, تھرڈ, پارٹی, ڈٹ, سفارش]
15             [بٹ, کوائن, قیمت, ریکارڈ, سطح, پر, پہنچ]
16    [پیٹرول, قیمت, برقرار, ڈیزل, روپے, فی, لیٹر, م...
17    [ئل, کمپنیز, اور, ڈیلرز, مارجن, 16فیصد, تک

In [54]:
#Applying CountVectorizer
bow_transformer = CountVectorizer(analyzer=text_cleaning).fit(df['Headline'])
bow_transformer.vocabulary_

{'عالمی': 73,
 'بینک': 36,
 'عسکریت': 74,
 'پسندی': 120,
 'متاثرہ': 91,
 'خاندانوں': 48,
 'معاونت': 97,
 'کرے': 137,
 'گا': 149,
 'مالی': 90,
 'سال': 62,
 '2020': 2,
 'ریٹرن': 56,
 'فائل': 76,
 'والوں': 110,
 'تعداد': 38,
 '23': 3,
 'فیصد': 80,
 'کمی': 143,
 'جاپان': 45,
 'سندھ': 67,
 'خصوصی': 49,
 'اقتصادی': 19,
 'زون': 60,
 'سرمایہ': 64,
 'کاری': 135,
 'دعوت': 53,
 'برامدات': 28,
 '767': 6,
 'بڑھ': 34,
 'ارب': 11,
 '16': 0,
 'کروڑ': 136,
 'ڈالر': 129,
 'زائد': 58,
 'الیکٹرک': 21,
 'اضافی': 17,
 'بجلی': 26,
 'گیس': 154,
 'فراہمی': 77,
 'قانونی': 81,
 'تقاضے': 40,
 'تعطل': 39,
 'شکار': 69,
 'کھانے': 146,
 'پینے': 125,
 'اشیا': 15,
 'قیمتیں': 85,
 'بلند': 32,
 'ترین': 37,
 'سطح': 65,
 'پر': 119,
 'پہنچ': 123,
 'گئیں': 148,
 'اقوام': 20,
 'متحدہ': 92,
 'صنعتی': 71,
 'صارفین': 70,
 'کیلئے': 147,
 'پیک': 127,
 'ور': 111,
 'ٹیرف': 114,
 'اسکیم': 14,
 'خاتمہ': 47,
 'اور': 23,
 'سبسڈی': 63,
 'منظور': 102,
 'پاکستان': 118,
 'موبائل': 103,
 'کمپنیاں': 141,
 'مقامی': 100,
 'طور': 72,
 'اسمبلنگ':

In [55]:
#Showing Vector with Index

title_bow = bow_transformer.transform(df['Headline'])
print(title_bow)

  (0, 36)	1
  (0, 48)	1
  (0, 73)	1
  (0, 74)	1
  (0, 91)	1
  (0, 97)	1
  (0, 120)	1
  (0, 137)	1
  (0, 149)	1
  (1, 2)	1
  (1, 3)	1
  (1, 38)	1
  (1, 56)	1
  (1, 62)	1
  (1, 76)	1
  (1, 80)	1
  (1, 90)	1
  (1, 110)	1
  (1, 143)	1
  (2, 19)	1
  (2, 45)	1
  (2, 49)	1
  (2, 53)	1
  (2, 60)	1
  (2, 64)	1
  :	:
  (18, 78)	1
  (18, 90)	1
  (18, 101)	1
  (18, 103)	1
  (18, 112)	1
  (18, 119)	1
  (18, 134)	1
  (19, 12)	1
  (19, 25)	1
  (19, 31)	1
  (19, 35)	1
  (19, 51)	1
  (19, 94)	1
  (19, 99)	1
  (19, 122)	1
  (19, 138)	1
  (20, 16)	1
  (20, 23)	1
  (20, 27)	1
  (20, 50)	1
  (20, 96)	1
  (20, 109)	1
  (20, 115)	1
  (20, 121)	1
  (20, 147)	1


In [56]:
tfidf_transformer = TfidfTransformer().fit(title_bow)
print(tfidf_transformer)
title_tfidf = tfidf_transformer.transform(title_bow)
print(title_tfidf)
print("shape = ", title_tfidf.shape)

TfidfTransformer()
  (0, 149)	0.3375685826755092
  (0, 137)	0.3375685826755092
  (0, 120)	0.3375685826755092
  (0, 97)	0.3375685826755092
  (0, 91)	0.3375685826755092
  (0, 74)	0.3375685826755092
  (0, 73)	0.3375685826755092
  (0, 48)	0.3375685826755092
  (0, 36)	0.29728709343592985
  (1, 143)	0.30041133371461554
  (1, 110)	0.34111614792842515
  (1, 90)	0.30041133371461554
  (1, 80)	0.2715308084175545
  (1, 76)	0.34111614792842515
  (1, 62)	0.2715308084175545
  (1, 56)	0.34111614792842515
  (1, 38)	0.34111614792842515
  (1, 3)	0.34111614792842515
  (1, 2)	0.30041133371461554
  (2, 135)	0.35355339059327373
  (2, 67)	0.35355339059327373
  (2, 64)	0.35355339059327373
  (2, 60)	0.35355339059327373
  (2, 53)	0.35355339059327373
  (2, 49)	0.35355339059327373
  :	:
  (18, 78)	0.3073609565140677
  (18, 62)	0.24466144304540285
  (18, 54)	0.20798457885277286
  (18, 52)	0.3073609565140677
  (18, 11)	0.24466144304540285
  (18, 5)	0.3073609565140677
  (18, 2)	0.2706840923214377
  (19, 138)	0.333333

In [57]:
# from sklearn.model_selection import train_test_split
X= title_tfidf
y = df['Headline']

In [58]:
#elmo = hub.load("https://tfhub.dev/google/elmo/2", trainable=True)
elmo = MultinomialNB().fit(X,y)

In [59]:
#Prediction
all_predictions = elmo.predict(X)
print(all_predictions)

['عالمی بینک عسکریت پسندی سے متاثرہ خاندانوں کی معاونت کرے گا'
 'مالی سال 2020 ریٹرن فائل کرنے والوں کی تعداد میں 23 فیصد کمی'
 'جاپان کو سندھ کے خصوصی اقتصادی زون میں سرمایہ کاری کی دعوت'
 'برامدات 767 فیصد بڑھ کر ارب 16 کروڑ ڈالر سے زائد'
 'کے الیکٹرک کو اضافی بجلی گیس کی فراہمی کے قانونی تقاضے تعطل کا شکار'
 'کھانے پینے کی اشیا کی قیمتیں سال کی بلند ترین سطح پر پہنچ گئیں اقوام متحدہ'
 'صنعتی صارفین کیلئے بجلی کے پیک ور ٹیرف اسکیم کا خاتمہ اور سبسڈی منظور'
 'پاکستان میں موبائل کمپنیاں مقامی طور پر اسمبلنگ کی جانب گامزن'
 'گنے کی بروقت کٹائی سے چینی کی قیمت کم ہو کر 85 روپے فی کلوگرام تک پہنچ گئی'
 'نومبر میں مہنگائی معمولی کمی سے 83 فیصد رہی'
 'ڈسکوز کا بجلی کے ٹیرف میں 86 پیسے اضافے کا مطالبہ'
 'جوہری بجلی گھر کی زمائش کیلئے ایندھن کی لوڈنگ کا غاز'
 'پاک سوزوکی اور انڈس موٹرز کی گاڑیوں کی قیمتوں میں ایک لاکھ روپے تک اضافہ'
 'نومبر میں ریونیو کی وصولی 346 ارب روپے کے ساتھ ہدف سے قریب رہی'
 'نیشنل بینک میں تقرریوں کے تھرڈ پارٹی ڈٹ کی سفارش'
 'بٹ کوائن کی قیمت ریکارڈ سطح پر پہنچ گئی'
 

In [62]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y,all_predictions)



array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,